In [15]:
import os
import numpy as np
import argparse
import cv2
import matplotlib.pyplot as plt
import time
from search_color_test import *
import hyperopt

In [12]:
class color_optmizer:
    def __init__(self):
        self.TP = [0,0,0,0,0,0,0]
        self.FP = [0,0,0,0,0,0,0]
        self.TN = [0,0,0,0,0,0,0]
        self.FN = [0,0,0,0,0,0,0]
        self.GT = [0,0,0,0,0,0,0]
        self.colors = ['red','yellow','blue','green'.'white','black','gray']
        
    def evaluate(self,color,Dataroot = '/root/ND_classifier/color_data/Vehicle_color_49466'):
        ROOT = Dataroot
        idx = self.colors.index(color)
        for root, subroot, fnames in os.walk(ROOT):
            for fname in fnames:
                target = os.path.join(root,fname)
                label = target.split("/")[5].split("_")[1]
                if (int(target.split("/")[5].split("_")[0])>17):
                    continue
                #print target.endswith('.jpg')
                if (target.endswith('.jpg')==False):
                    continue
                #print target , label
                im = cv2.imread(target)
                search_color = Search_Color(auto_update_status = False)
                res = search_color.color_detection_test(im)
                self.GT[self.colors.index(label)]+=1
                if (color == label):
                    if(label in res):
                        self.TP[idx]+=1
                    else:
                        self.FN[idx]+=1
                else:
                    if(color in res):
                        self.FP[idx]+=1
                    else:
                        self.TN[idx]+=1
        Tp = float(self.TP[idx])
        Tn = float(self.TN[idx])
        Fp = float(self.FP[idx])
        Fn = float(self.FN[idx])
        precision = Tp/(Tp+Fp)
        accuracy = (Tp+Tn)/(Tp+Tn+Fp+Fn)
        recall = Tp/(Tp+Fn)
        return [precision,accuracy,recall]
        
    def getColor(self,color):
        idx = self.colors.index(color)
        Tp = float(self.TP[idx])
        Tn = float(self.TN[idx])
        Fp = float(self.FP[idx])
        Fn = float(self.FN[idx])
        precision = Tp/(Tp+Fp)
        accuracy = (Tp+Tn)/(Tp+Tn+Fp+Fn)
        recall = Tp/(Tp+Fn)
        return [precision,accuracy,recall]
    
    

In [13]:
Optimizer = color_optmizer()

In [14]:
Optimizer.evaluate(color='black')

1 loop, best of 3: 2min 33s per loop


In [25]:
from hyperopt import hp

base_space = [
    hp.uniform('threshold',0,10), #threshold of detection      
    hp.uniform('roiScale',4,10), #roi scale   
    hp.uniform('filtratio',0.5,0.75),  #mask filter threshold
    
]
color_space_dict = {
    'color_space_black' : [  
        hp.uniform('black_h_lower',15,45),
        hp.uniform('black_h_upper',50,80),
        hp.uniform('black_s_lower', 60,120)       
    ],
    
    'color_space_red' : [
        hp.uniform('red_h_lower',140,160),
        hp.uniform('red_h_upper',20,40),
        hp.uniform('red_s_lower', 60,120),
        hp.uniform('red_v_lower',20,60)
    ],
    'color_space_yellow' : [
        hp.uniform('yellow_h_lower',0,25),
        hp.uniform('yellow_h_upper',20,60),
        hp.uniform('yellow_s_lower', 30,70),
        hp.uniform('yellow_v_lower',25,50),
    ],
    'color_space_blue' : [
        hp.uniform('blue_h_lower',70,95),
        hp.uniform('blue_h_upper',35,75),
        hp.uniform('blue_s_lower', 30,80),
        hp.uniform('blue_v_lower',115,145),
    ],
    'color_space_green' : [
        hp.uniform('green_h_lower',25,45),
        hp.uniform('green_h_upper',35,75),
        hp.uniform('green_s_lower', 30,80),
        hp.uniform('green_v_lower',75,100),
    ],
    'color_space_white' : [
        hp.uniform('white_h_lower',170,210),
        hp.uniform('white_h_upper',0,20),
        hp.uniform('white_s_lower', 160,210),
        hp.uniform('white_v_lower',15,60),
    ]   
}

colorset    = ['red','yellow','blue','green','white','black',]
color2opt   = ['red','yellow','blue','green','white','black']
log_prefix         = 'res_log.txt'

In [17]:
from __future__        import division
import os
import sys
import cv2
import json
import numpy as np
from matplotlib.colors import hsv_to_rgb
from PIL               import Image
from importlib         import *
from collections       import deque
from config            import *
from lib.auto_white_balance import auto_white_balance
from lib.lamp_filter        import lamp_filter
from lib.crop_aspect_ratio  import crop_aspect_ratio
from lib.get_roi            import get_roi
from lib.extract_feature    import extract_feature, train_kmean
from apscheduler.scheduler            import Scheduler
import datetime
import logging
import operator
from sklearn.externals import joblib
from sklearn import cluster,metrics

In [21]:
class Color_optimize:
    
    def __init__(self,isNight):
        self.isNight = isNight
        self.profile = {}
        self.classifier = joblib.load('kmeans.pkl')
        if self.classifier.cluster_centers_[0][0] > self.classifier.cluster_centers_[1][0]:
            self.darkCenter = [self.classifier.cluster_centers_[1]]
        else:
            self.darkCenter = [self.classifier.cluster_centers_[0]]
            
    def detectDark(self,img):
        feature = extract_feature(img)
        out = self.classifier.predict([feature])
        sample_feature = [[34.830490711038301, 0.34355039919243824, 0.1076548133377645]]
        if (out[0]==self.classifier.predict(sample_feature[0])):
            return 1
        else:
            return 0
        
    def detect(self,frame,threshold,color_param,color):
        filtRatio = [0.68,0.745,0.72,0.67,0.74,0.73]
        roiScale = [7.38,9.96,8.16,6.18,8.49,8.07]
        colors = ['red','yellow','blue','green','white','black']
        idx = colors.index(color)
        frameIsDark = 0
        if (detectDark(frame) == 1):
            frameIsDark = 1
        frame = crop_aspect_ratio(frame,True)
        if (frameIsDark == 1):
            frame = auto_white_balance(frame)

        roiWid = 10
        roiEdg = roiScale
               
        src_height, src_width, src_channels = frame.shape
        
        roiX = int(src_width / roiWid)
        roiWidth = int(roiX * roiEdg)
        roiY = int(src_height / roiWid)
        roiHeight = int(roiY * roiEdg)
        frame = frame[roiY : roiY+roiHeight, roiX : roiX+roiWidth]
        
        if (frameIsDark ==1):
            frame,lightMask = imgFilt(frame,filtratio)
            blockedPxl = sum(np.count_nonzero(e) for e in lightMask)
        else:
            blockedPxl = 0
        
        max_value = ((src_height*src_width)-blockedPxl)*255
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
            
        if color2opt is 'white':
            lower = np.array([0,0, color_param[0]])   
            upper = np.array([179, color_param[1], 255])
            mask = cv2.inRange(hsv, lower, upper)
            lower = np.array([0,color_param[1], color_param[2]])
            upper = np.array([179, color_param[3], 255])
            mask = mask + cv2.inRange(hsv, lower, upper)
            
        elif color2opt is 'black':
            lower = np.array([0,0,0])
            upper = np.array([179,color_param[0],color_param[1]])
            mask  = cv2.inRange(hsv,lower,upper)
            lower = np.array([0,color_param[0],0])
            upper = np.array([179,255,color_param[2]])
            mask  = mask + cv2.inRange(hsv, lower, upper)   
        else: #others
            lower = np.array([color_param[0],color_param[1], color_param[2]])   
            upper = np.array([color_param[3], 255, 255])
            mask = cv2.inRange(hsv, lower, upper)
        print mask
        
        if(float(max_value) != 0):
            Val = float(mask.sum()) / float(max_value)
        else:
            Val = 0
    
        res = round(Val,2)

        return res

In [28]:
import pickle
import time
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from pprint import pprint
import hyperopt.pyll.stochastic
from __future__ import division

# minimize the objective over the space
from hyperopt import fmin, tpe
import hyperopt
import sys
import time

def restart_line():
    sys.stdout.write('\r')
    sys.stdout.flush()


    
min = 1
state = ''
# define an objective function
def objective(args):
    global min
    global state
    global color
    global log
    
    
    threshold, roiScale, filtratio = args[0:3]
    color_param = args[3:]
    color_param = [int(param) for param in color_param]
    
    thr = 0.1 + threshold * 0.05
    
    total,this_color_count,fp,fn = opt_color(thr, roiScale, filtratio, color_param,color)
                    
    tp = total - fp - fn
    beta = 5
    
    f1_score = float(((1 + beta**2) * tp)) / ((1 + beta**2)*tp + beta**2 * fn + fp)
    
    if(1 - f1_score < min):
        min = 1 - f1_score
        state = {
            'color'      :   color,
            'Total_frame':   total,
            'Color_frame':   this_color_count,
            'f1_score':      round(1-f1_score,3),
            'fp':            round(fp/(total-this_color_count),3),
            'fn':            round(fn/this_color_count,3),
            'thr':           thr,
            'roiScale':      roiScale,
            'filtratio':     filtratio,
            'color_param':   color_param,
            
        }
        with open(color + '_' + log_prefix, "a") as res_log:
            pprint(state, res_log)
        pprint(state)
        print('\n')

    return {
        'loss': 1-f1_score,
        'status': STATUS_OK,
        'eval_time':time.time()
    }

test_search_color = Color_optimize(True)

color = 'red'
space = base_space + color_space_dict['color_space_' + color]
trials = Trials()
best = fmin(objective, space, algo=tpe.suggest, max_evals=6000,trials= trials)